In [1]:
import pandas as pd

df = pd.read_csv("../data/wrime.tsv", sep="\t")

columns = [
    "Sentence",
    "Avg. Readers_Joy",
    "Avg. Readers_Sadness",
    "Avg. Readers_Anticipation",
    "Avg. Readers_Surprise",
    "Avg. Readers_Anger",
    "Avg. Readers_Fear",
    "Avg. Readers_Disgust",
    "Avg. Readers_Trust",
]

train_df = df[df["Train/Dev/Test"] == "train"][columns]
val_df = df[df["Train/Dev/Test"] == "dev"][columns]
test_df = df[df["Train/Dev/Test"] == "test"][columns]

print("訓練用データ", len(train_df))
print("検証用データ", len(val_df))
print("評価用データ", len(test_df))

train_df.to_csv("../data/train.tsv", sep="\t", index=False)
val_df.to_csv("../data/val.tsv", sep="\t", index=False)
test_df.to_csv("../data/test.tsv", sep="\t", index=False)

/Users/nakagawatsubasa/.pyenv/versions/sentiment-analysis/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


訓練用データ 40000
検証用データ 1200
評価用データ 2000
